In [ ]:
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from matplotlib.pyplot import plot


In [ ]:
#load dataset
fn2 = "log_test_MNLI_entropy_opt_12class.txt"
fn = "log_train_MNLI_entropy_opt_12class.txt"

nparr2 = np.loadtxt(fn2, delimiter=",")
nparr = np.loadtxt(fn, delimiter=",")

In [ ]:
#split 80/20
#whether to average entropies up to start_layer or just use entropy at start layer
avg = 0 
#layer after which you make entropy prediction
start_layer = 6 #indexed 0-11

#split = round(0.95*nparr.shape[0])
#train = nparr[:split,:]
#test = nparr[split:,:]
test = nparr2[:,:11]
train = nparr[:,:11]

if avg:
  (x_train, y_train) = (train[:,:start_layer], train[:,start_layer+1:])
  (x_test, y_test) = (test[:,:start_layer], test[:,start_layer+1:])
  x_train = np.mean(x_train, axis=-1)
  x_test = np.mean(x_test, axis=-1)
else:
  (x_train, y_train) = (train[:,start_layer], train[:,start_layer+1:])
  (x_test, y_test) = (test[:,start_layer], test[:,start_layer+1:])

In [ ]:
# E1 -> E2 E3 E4 E5 E6 E7 E8 E9 E10 E11

model = Sequential()
model.add(Dense(64, activation='relu', input_dim = 1)) #input layer
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
#model.add(Dense(64, activation='linear'))
model.add(Dense(10-start_layer)) # output layer

model.compile(
    optimizer='adam',
    loss='mean_squared_error' 
)

model.fit(x_train, y_train, epochs=3, verbose=1)

Epoch 1/3
12268/12268 [==============================] - 19s 2ms/step - loss: 0.0497
Epoch 2/3
12268/12268 [==============================] - 19s 2ms/step - loss: 0.0484
Epoch 3/3
12268/12268 [==============================] - 19s 2ms/step - loss: 0.0483


In [ ]:
#check prediction

val_loss = model.evaluate(x_test, y_test)
#val_loss = model.evaluate(x_train,y_train)
#model.summary()


#i=7
#x = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
#while(i < 10):
#inp = x_test[i]
#out = y_test[i]
#pred = model.predict(np.array([inp]), batch_size=1)
#print("prediction:")
#print(pred[0])
#print("output:")
#print(out)
#i = i + 1
#print(inp)
#plot(x, out)
#plot(x, np.transpose(pred))

  

307/307 [==============================] - 0s 924us/step - loss: 0.0505


In [ ]:
#produce lookup table
table = np.zeros((100, 11-start_layer))
for i in range(0,100):
  xval = i*0.01;
  yval = model.predict(np.array([xval]), batch_size=1)
  table[i,0] = xval
  table[i,1:] = yval[0]

threshold_indices = table < 0
table[threshold_indices] = 0

np.savetxt("mnli_lookup_table_opt_layer7.csv", table, delimiter=",")
print(table)

[[0.         0.02224375 0.0159677  0.01438424 0.01826167]
 [0.01       0.02224375 0.0159677  0.01438424 0.01826167]
 [0.02       0.02224375 0.0159677  0.01438424 0.01826167]
 [0.03       0.02238394 0.01607253 0.01447886 0.01834077]
 [0.04       0.02760141 0.01997368 0.01799999 0.02128394]
 [0.05       0.03281888 0.02387486 0.02152111 0.02422711]
 [0.06       0.03936858 0.02877179 0.02593736 0.0279176 ]
 [0.07       0.04595281 0.03370483 0.03049038 0.03174715]
 [0.08       0.05253609 0.03865106 0.03519636 0.035738  ]
 [0.09       0.0591194  0.04359733 0.03990234 0.03972886]
 [0.1        0.06570272 0.04854359 0.04460832 0.04371972]
 [0.11       0.07228601 0.05348983 0.04931431 0.04771059]
 [0.12       0.0788693  0.05843608 0.05402029 0.05170144]
 [0.13       0.08535373 0.06331113 0.05868969 0.05566829]
 [0.14       0.09333654 0.06896409 0.06450999 0.06065941]
 [0.15       0.10160741 0.07477289 0.07053909 0.06583366]
 [0.16       0.10971011 0.08039175 0.075577   0.06998681]
 [0.17       0